In [3]:
import json

import pandas as pd

In [4]:
df = pd.read_csv('../../data/raw/train.csv')
df

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
...,...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,A,False,"[[-8.612469,41.14602],[-8.612487,41.145993],[-..."
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,A,False,"[[-8.610138,41.140845],[-8.610174,41.140935],[..."
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,A,False,[]
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,A,False,"[[-8.630712,41.154885],[-8.63073,41.154813],[-..."


In [3]:
from pandarallel import pandarallel
import json
pandarallel.initialize(progress_bar=True)
df['POLYLINE'] = df['POLYLINE'].parallel_apply(json.loads)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
df.to_csv("../data/train-polyline-list.csv", index=False)

In [5]:
df['polyline_length'] = df['POLYLINE'].apply(len)
longest_list_row = df.loc[df['polyline_length'].idxmax()]
longest_ride = df[df['TRIP_ID'] == longest_list_row['TRIP_ID']]

In [1]:
longest_ride = longest_ride.explode('POLYLINE', ignore_index=True)
longest_ride

NameError: name 'longest_ride' is not defined

In [9]:
longest_ride['datetime'] = pd.to_datetime(longest_ride['TIMESTAMP'], unit='s')
longest_ride['adjusted_time'] = longest_ride['datetime'] + pd.to_timedelta(longest_ride.index * 15, unit='s')
longest_ride

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,polyline_length,datetime,adjusted_time
0,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.583435, 41.158665]",3881,2014-02-22 09:27:55,2014-02-22 09:27:55
1,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.582958, 41.158899]",3881,2014-02-22 09:27:55,2014-02-22 09:28:10
2,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.582553, 41.159817]",3881,2014-02-22 09:27:55,2014-02-22 09:28:25
3,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.581986, 41.161293]",3881,2014-02-22 09:27:55,2014-02-22 09:28:40
4,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.582607, 41.162805]",3881,2014-02-22 09:27:55,2014-02-22 09:28:55
...,...,...,...,...,...,...,...,...,...,...,...,...
3876,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.583795, 41.163201]",3881,2014-02-22 09:27:55,2014-02-23 01:36:55
3877,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.584146, 41.163201]",3881,2014-02-22 09:27:55,2014-02-23 01:37:10
3878,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.58411, 41.16321]",3881,2014-02-22 09:27:55,2014-02-23 01:37:25
3879,1393061275620000681,C,NaN,NaN,20000681,1393061275,A,False,"[-8.584074, 41.163228]",3881,2014-02-22 09:27:55,2014-02-23 01:37:40


In [10]:
longest_ride.to_csv("../data/longest-ride-list.csv", index=False)

In [12]:
print(longest_ride.head(1))
print(longest_ride.tail(1))

               TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0  1393061275620000681         C          NaN           NaN  20000681   

    TIMESTAMP DAY_TYPE  MISSING_DATA                POLYLINE  polyline_length  \
0  1393061275        A         False  [-8.583435, 41.158665]             3881   

             datetime       adjusted_time  
0 2014-02-22 09:27:55 2014-02-22 09:27:55  
                  TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
3880  1393061275620000681         C          NaN           NaN  20000681   

       TIMESTAMP DAY_TYPE  MISSING_DATA                POLYLINE  \
3880  1393061275        A         False  [-8.584074, 41.163246]   

      polyline_length            datetime       adjusted_time  
3880             3881 2014-02-22 09:27:55 2014-02-23 01:37:55  


In [7]:
no_missing_data = df[(df["MISSING_DATA"] == "FALSE") & (df["POLYLINE"] is not None)]

In [ ]:
no_missing_data['polyline_length'] = no_missing_data['POLYLINE'].apply(len)
longest_list_row = df.loc[df['polyline_length'].idxmax()]
longest_ride = df[df['TRIP_ID'] == longest_list_row['TRIP_ID']]

In [8]:
shortest_list_row = no_missing_data.loc[no_missing_data['polyline_length'].idxmin()]
shortest_ride = no_missing_data[no_missing_data['TRIP_ID'] == shortest_list_row['TRIP_ID']]

KeyError: 'polyline_length'

In [18]:
shortest_ride = shortest_ride.explode('POLYLINE', ignore_index=True)
shortest_ride

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,polyline_length,datetime,adjusted_time
0,1372665673620000353,C,NaN,NaN,20000353,1372665673,A,False,NaN,0,2013-07-01 08:01:13,2013-07-01 11:11:43
